<a href="https://colab.research.google.com/github/pangeab-blip/EvGeo-Exercises/blob/main/Ka-Boom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Extended impact + crater model (Chicxulub-calibrated K)
# Riferimenti chiave usati per la calibrazione:
# - D_finale Chicxulub ~180–200 km (LPI/USRA)
# - Angolo d'impatto ~60° rispetto all'orizzontale (Nature 2020; sintesi divulgative)
# - Impattatore ~10–12 km; target: piattaforma carbonatica-evaporitica con acqua superficiale
# Nota: questi vincoli fisici fissano K ~12–15 nel modello monostadio usato qui.

import numpy as np
import math
import matplotlib.pyplot as plt

try:
    from ipywidgets import interact, FloatSlider, Dropdown, fixed
except Exception:
    interact = None  # Colab in modalità "no widgets"

# ---- Costanti e conversioni
G_PER_CM3_TO_KG_PER_M3 = 1000.0
KM_TO_M  = 1000.0
KMS_TO_MS = 1000.0
J_PER_KT_TNT = 4.184e12
J_PER_MT_TNT = 4.184e15
HIROSHIMA_KT = 15.0
J_PER_HIRO = HIROSHIMA_KT * J_PER_KT_TNT

# ---- Target planetari (densità media del "target" intesa come roccia bersaglio)
TARGETS = {
    "Moon":   {"g": 1.62, "rho_t": 2.7},
    "Mercury":{"g": 3.70, "rho_t": 3.0},
    "Mars":   {"g": 3.71, "rho_t": 2.7},
    "Ceres":  {"g": 0.27, "rho_t": 2.2},
    "Venus":  {"g": 8.87, "rho_t": 2.9},
    "Earth":  {"g": 9.81, "rho_t": 2.7},  # piattaforma carbonatica ~2.7 g/cm3
}

def crater_diameter_monostage(rho_i_gcm3, rho_t_gcm3, g_mps2, v_kms, theta_deg_from_horizontal,
                              D_imp_km, K_const):
    """
    Formula monostadio: D_f = K * (rho_i/rho_t)^(1/3) * sin(theta)^(1/3) * g^(-0.22) * v^(0.44) * D_imp^(0.78)
    theta definito rispetto all'orizzontale.
    """
    rho_ratio = (rho_i_gcm3 / max(rho_t_gcm3, 1e-9)) ** (1.0 / 3.0)
    sin_term = max(math.sin(math.radians(theta_deg_from_horizontal)), 1e-9) ** (1.0 / 3.0)
    g_term = g_mps2 ** (-0.22)
    v_term = v_kms ** 0.44
    D_term = D_imp_km ** 0.78
    return K_const * rho_ratio * sin_term * g_term * v_term * D_term

def calibrate_K_to_observation(D_final_obs_km,
                               rho_i_gcm3=3.0, rho_t_gcm3=2.7,
                               g_mps2=9.81, v_kms=20.0, theta_deg_from_horizontal=60.0,
                               D_imp_km=12.0):
    """
    Ricava K imponendo D_f osservato (es. Chicxulub ~190 km).
    """
    # Denominatore della relazione
    rho_ratio = (rho_i_gcm3 / max(rho_t_gcm3, 1e-9)) ** (1.0 / 3.0)
    sin_term = max(math.sin(math.radians(theta_deg_from_horizontal)), 1e-9) ** (1.0 / 3.0)
    g_term = g_mps2 ** (-0.22)
    v_term = v_kms ** 0.44
    D_term = D_imp_km ** 0.78
    denom = rho_ratio * sin_term * g_term * v_term * D_term
    return D_final_obs_km / denom

# ---- Calibrazione su Chicxulub (valori centrali)
D_obs_ref = 190.0
K_chicx = calibrate_K_to_observation(
    D_final_obs_km=D_obs_ref,
    rho_i_gcm3=3.0, rho_t_gcm3=2.7,
    g_mps2=9.81, v_kms=20.0, theta_deg_from_horizontal=60.0,
    D_imp_km=12.0
)
# Per robustezza, imposto il default K_ref = 13.0 (all'interno di 12–15 tipico della calibrazione)
K_ref = float(np.clip(K_chicx, 12.0, 15.0))

def crater_extended(
    rho_gcm3=3.0,
    D_km=12.0,
    v_kms=20.0,
    theta_deg=60.0,
    target="Earth",
    rho_t_gcm3=None,
    K_const=K_ref,
    v_min_kms=5.0, v_max_kms=70.0,
    D_min_km=0.1, D_max_km=500.0
):
    # Proprietà impattatore
    rho = rho_gcm3 * G_PER_CM3_TO_KG_PER_M3
    R = 0.5 * D_km * KM_TO_M
    V = (4/3)*math.pi*R**3
    m = rho * V
    v = v_kms * KMS_TO_MS
    E = 0.5 * m * v**2
    Mt = E / J_PER_MT_TNT
    hiro = E / J_PER_HIRO

    # Proprietà target
    g = TARGETS[target]["g"]
    rho_t = rho_t_gcm3 if (rho_t_gcm3 and rho_t_gcm3>0) else TARGETS[target]["rho_t"]

    # Calcolo cratere
    D_crater_km = crater_diameter_monostage(
        rho_i_gcm3=rho_gcm3, rho_t_gcm3=rho_t, g_mps2=g,
        v_kms=v_kms, theta_deg_from_horizontal=theta_deg, D_imp_km=D_km, K_const=K_const
    )

    print("=== Crater model (Chicxulub-calibrated K) ===")
    print(f"Target: {target} | g = {g:.2f} m/s² | ρ_t = {rho_t:.2f} g/cm³")
    print(f"Impactor: ρ = {rho_gcm3:.2f} g/cm³, D = {D_km:.2f} km, v = {v_kms:.2f} km/s, θ = {theta_deg:.1f}° (da orizzontale)")
    print(f"Energy: {E:.3e} J → {Mt:.3e} Mt TNT (~{hiro:.3e} Hiroshima)")
    print(f"K (in uso) = {K_const:.3f}  |  K_Chicxulub_centrale (auto) = {K_chicx:.3f}")
    print(f"Stima diametro cratere finale ≈ {D_crater_km:.1f} km")
    if D_crater_km > 300:
        print("Avvertenza: >300 km entra nel regime 'basin'; la legge di potenza è solo indicativa.")

    # Grafico 1: diametro cratere vs. diametro impattatore
    Dgrid_km = np.geomspace(max(D_min_km, 0.05), max(D_max_km, D_km*1.2), 300)
    Dcr_grid = crater_diameter_monostage(rho_gcm3, rho_t, g, v_kms, theta_deg, Dgrid_km, K_const)
    plt.figure(figsize=(6,4))
    plt.plot(Dgrid_km, Dcr_grid)
    plt.scatter([D_km], [D_crater_km], s=40)
    plt.xscale("log"); plt.yscale("log")
    plt.xlabel("Diametro impattatore (km)")
    plt.ylabel("Diametro cratere finale (km)")
    plt.title("Crater size vs impactor size (K calibrato su Chicxulub)")
    plt.grid(True, linestyle=":")
    plt.show()

    return {"mass_kg": m, "energy_J": E, "energy_MtTNT": Mt, "crater_diam_km": D_crater_km}

# ---- Widget interattivo (se disponibile)
if interact is not None:
    interact(
        crater_extended,
        rho_gcm3=FloatSlider(value=3.0, min=0.5, max=8.0, step=0.1, description="ρ_i (g/cm³)"),
        D_km=FloatSlider(value=12.0, min=0.5, max=200.0, step=0.5, description="D_imp (km)"),
        v_kms=FloatSlider(value=20.0, min=5.0, max=40.0, step=0.5, description="v (km/s)"),
        theta_deg=FloatSlider(value=60.0, min=5.0, max=85.0, step=1.0, description="θ (deg, da orizz.)"),
        target=Dropdown(options=list(TARGETS.keys()), value="Earth", description="Target"),
        rho_t_gcm3=FloatSlider(value=2.7, min=1.5, max=3.5, step=0.01, description="ρ_t override"),
        K_const=FloatSlider(value=K_ref, min=8.0, max=20.0, step=0.05, description="K (calibrato)"),
        v_min_kms=fixed(5.0), v_max_kms=fixed(40.0),
        D_min_km=fixed(0.5), D_max_km=fixed(500.0),
    )

# ---- Utility: ricalibra K su qualunque set osservativo
def fit_K_from_observation(D_final_obs_km, D_imp_km, v_kms, theta_deg_from_horizontal,
                           rho_i_gcm3=3.0, rho_t_gcm3=2.7, g_mps2=9.81):
    """
    Dati osservabili (D_f, D_i, v, theta, densità), ritorna K necessario a riprodurre D_f.
    """
    return calibrate_K_to_observation(D_final_obs_km, rho_i_gcm3, rho_t_gcm3,
                                      g_mps2, v_kms, theta_deg_from_horizontal, D_imp_km)

# Esempio numerico (Chicxulub centrale):
# K_fit = fit_K_from_observation(190.0, 12.0, 20.0, 60.0, 3.0, 2.7, 9.81)
# print(K_fit)

interactive(children=(FloatSlider(value=3.0, description='ρ_i (g/cm³)', max=8.0, min=0.5), FloatSlider(value=1…